---
# 환경 설정
---

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/aiffel/NLP

/content/drive/MyDrive/aiffel/NLP


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [39]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [11]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [14]:
# GPU 사용
device = torch.device("cuda:0")

In [15]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/drive/MyDrive/aiffel/NLP/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/drive/MyDrive/aiffel/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


---
# 데이터셋 전처리
---

In [147]:
dataset = pd.read_csv('/content/drive/MyDrive/aiffel/NLP/dataset/korea_emotion_dataset.csv')

- 분노 = 0 
- 행복 = 1 
- 혐오 = 2 
- 두려움 = 3 
- 중립 = 4 
- 슬픔 = 5
- 놀라움 = 6

In [148]:
dataset.head(15)

,text1,text2,text3
0,개를 예쁘다고 사놓고 끝까지 키우지도 않고 버리는 사람들이 엄청 많아졌대.,2,emotion_10_gjs_v1
1,지금도 그대로 있어. 치우는 사람이 없어.,2,emotion_10_gjs_v1
2,맞아. 무기력증인 것 같아. 한동안 정말 바빴었거든.,5,emotion_10_gjs_v1
3,오늘이 발표날인데 연락이 없더라고. 그래서 알아봤더니 명단에 내 이름이 없대.,5,emotion_10_gjs_v1
4,그치. 개 키우는 사람이 늘어나니까 그만큼 버리는 사람도 늘어나는 거야!,2,emotion_10_gjs_v1
5,공채로 볼 수 있는 마지막 회사였어. 그래서 정말 많이 노력했거든.,5,emotion_10_gjs_v1
6,버려진 개가 워낙 많으니까 시설이 부족할 수 밖에 없고 당연히 상황이 너무 열악하지.,2,emotion_10_gjs_v1
7,나 면접 또 떨어졌어.,5,emotion_10_gjs_v1
8,부모님께서 기대가 너무 크셔서 실망도 크실까봐 아직 말씀 못 드렸어.,5,emotion_10_gjs_v1
9,그렇지! 아 나 진짜 너무 싫어 그런 사람들. 대체 생명이 있는 개를 뭐라고 생각하...,2,emotion_10_gjs_v1


In [143]:
# n개의 데이터셋 랜덤으로 출력
dataset.sample(n=10)

,text1,text2,text3
8237,사람들이 귀여울 때 잠깐 키우다가 버리는 경우가 대부분이래. 정 들었을 텐데 어떻게...,2,emotion_10_gjs_v1
20711,좋은 생각인데 나는 말을 잘 못 해.\t,2,emotion_10_gjs_v1
24825,"그래, 좋아. 언제 우리집 한 번 놀러 와.",1,emotion_10_gjs_v1
31377,어. 해피가 많이 아파했던 것 같아.,5,emotion_10_gjs_v1
12472,함께 달리는 크루가 있어서 힘들 때마다 큰 힘이 되어 줘.,1,emotion_10_gjs_v1
7439,3년 전에 그냥 소개팅으로 만났지 뭐.,5,emotion_10_gjs_v1
19506,해피가 막 뛰어가는데 다행히 다친 사람은 없었어.,6,emotion_10_gjs_v1
36574,"아, 나 룸메이트랑 또 싸웠어.",0,emotion_10_gjs_v1
12434,내일 비 소식이 있어? 비가 많이 온대? 우산은 사무실에 없는데.,4,emotion_10_gjs_v1
27146,15년 동안이나 키워서 정이 많이 들었나봐. 고마워.,5,emotion_10_gjs_v1


In [151]:
dataset = dataset.sample(frac=1).reset_index(drop=True)

In [152]:
dataset

,text1,text2,text3
0,그 새끼는 할 일 없으면 집에나 갈 것이지 왜 괴롭혀?,0,emotion_10_gjs_v1
1,15년 동안이나 키워서 정이 많이 들었나봐. 고마워.,5,emotion_10_gjs_v1
2,다들 괜찮은 거 같아.,3,emotion_10_gjs_v1
3,응. 했는데 안고쳐지더라.,0,emotion_10_gjs_v1
4,너무 가고 싶어요!,1,emotion_10_gjs_v1
...,...,...,...
37167,당연히 그럴 때마다 화를 내 봤지. 하지만 서로 싸우기만 해. 입장 차는 좁혀지지가...,0,emotion_10_gjs_v1
37168,"여행도 가고 싶은데, 지금은 갈 수가 없어.",1,emotion_10_gjs_v1
37169,음악? 뭐 기분 좋아질만 한 음악이 있을까?,5,emotion_10_gjs_v1
37170,환기는 계속 시켰지!,0,emotion_10_gjs_v1


In [153]:
# 길이 확인 
len(dataset)

37172

In [154]:
data_list = [] # 최종 데이터셋 리스트 
for q, label in zip(dataset['text1'], dataset['text2']) :
  data = []  # 라벨과 내용이 하나의 리스트에 넣기 위해서 빈 리스트 생성
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [155]:
print((data_list[2]))
print((data_list[156]))
print((data_list[9257]))
print((data_list[7842]))


['다들 괜찮은 거 같아.', '3']
['밥 보다는 술이 마시고 싶은데, 지금 술 마시면 너무 안 좋을 것 같아.', '5']
['니가 진심으로 축하해주니까 내 기분이 너무 좋아.', '1']
['너무 놀랬어! 기절하는 줄?', '2']


---
# 데이터셋 나누기
----

In [156]:
train_data, test_data = train_test_split(data_list, test_size = 0.25, random_state = 53)

In [157]:
train_data[:5]

[['완전. 내가 너무 비싸가지고 못 사던 향순데, 당첨되니까 얼마나 기분 좋게요!', '1'],
 ['엊그제 갑자기 의식을 잃더니 조금 밥도 먹더니만 결국 죽고 말았어.', '5'],
 ['얼마나 됐다고 회사에서 연락이 또 왔어, 또!', '0'],
 ['일주일 내내 드시는 것 같아.', '0'],
 ['자취방 엘리베이턴데 정전인가봐.', '3']]

In [158]:
test_data[:5]

[['코로나로 인해서 재택 근무를 시작하면서 집 안에만 있으니까 마음이 울적해.', '5'],
 ['이직할려면 최소한 2년은 다녀야 돼. 아직 얼마 안됐잖아.', '0'],
 ['고등학교 동창이었어. 3년 내내 같이 붙어다녔어. 그 정도로 친했었다고.', '0'],
 ['집 안에 갇히게 돼서 너무 두려워.', '3'],
 ['당연히 말씀 드려야지! 다시는 그렇게 행동 못 하게 혼쭐을 내야 해!', '2']]

In [159]:
print(len(train_data))
print(len(test_data))

27879
9293


----
# Kobert로 입력 데이터 만들기 
---

- 버트는 입력 문장에 대한 요구 리스트

  - 각 토큰의 Vocabulary 인덱스를 추출해 이를 정해진 길이의 벡터로 생성.
  - 두 문장 혹은 하나의 문장이 들어올 수 있기 때문에 이들을 구분하기 위한 토큰 타입 벡터 생성
  - 유효 길이 벡터

- gluonNLP는 위 과정을 해주는 함수

In [160]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length = max_len, pad = pad, pair = pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self,i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [161]:
# 하이퍼파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [162]:
# 토큰화
tokenizer = get_tokenizer() # 파이토치 토큰화 방식
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False )

data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/MyDrive/aiffel/NLP/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


- 첫번째 출력값 = 패딩된 시퀀스 
- 두번째 출력값 = 길이와 타입
- 세번째 출력값 = 어텐션 마스크 시퀀스   
  어텐션 마스크 시퀀스는 1로 패딩된 값들은 연산할 필요가 없기 때문에  
  연산을 하지 않아도 된다고 알려주는 데이터

In [163]:
data_train[0]

(array([   2, 3461,  517,   54, 1435, 1458, 2514, 6752, 5330, 7321, 2086,
        2573, 5842, 5032, 6643, 5850,  517,   46, 1629, 5886, 5771, 3253,
        1282, 4204, 5400, 6999,  517,    5,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(29, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [164]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

---
# koBERT 학습 모델 만들기
---

In [165]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [166]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

---
# koBERT 모델 학습 시키기
---

In [167]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/436 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8641448020935059 train acc 0.21875
epoch 1 batch id 201 loss 0.08661903440952301 train acc 0.814443407960199
epoch 1 batch id 401 loss 0.32804495096206665 train acc 0.8739479426433915
epoch 1 train acc 0.8777622177134792


  0%|          | 0/146 [00:00<?, ?it/s]

epoch 1 test acc 0.943707191780822


  0%|          | 0/436 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9523923208640674


  0%|          | 0/436 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.09687689691781998 train acc 0.953125
epoch 5 batch id 201 loss 0.0034031374379992485 train acc 0.9874067164179104
epoch 5 batch id 401 loss 0.07556533068418503 train acc 0.9885052992518704
epoch 5 train acc 0.988639621559633


  0%|          | 0/146 [00:00<?, ?it/s]

epoch 5 test acc 0.9535695468914647


---
# 모델 저장 및 불러오기
---

In [168]:
torch.save(model, 'koBERT_emotion2.pt')

In [169]:
model = torch.load('koBERT_emotion2.pt')

---
# 새로운 문장으로 확인해보기
---

In [235]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("분노가")
            elif np.argmax(logits) == 1:
                test_eval.append("행복이")
            elif np.argmax(logits) == 2:
                test_eval.append("혐오가")
            elif np.argmax(logits) == 3:
                test_eval.append("두려움이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 6:
                test_eval.append("놀람이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.",'\n\n' ,np.round(logits,5),
              '\n |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |'  )

----

In [171]:
import warnings
warnings.filterwarnings('ignore')

In [236]:
predict('벌레 너무 싫어')

>> 입력하신 내용에서 혐오가 느껴집니다. 

 [ 0.61046 -1.31357  6.61053 -1.52915 -1.12763 -1.59571 -1.50455] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [237]:
predict('유령의 집은 너무 무서워')

>> 입력하신 내용에서 두려움이 느껴집니다. 

 [-0.99745 -1.76573 -1.64363  6.46971 -1.0115  -1.68035 -0.64799] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [238]:
predict('너 떄문에 화가 난다')

>> 입력하신 내용에서 분노가 느껴집니다. 

 [ 6.47124 -2.26495 -0.55912 -1.94558  0.71167 -1.22975 -0.98499] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [239]:
predict('이 멍청한 모델아')

>> 입력하신 내용에서 혐오가 느껴집니다. 

 [-0.873   -1.2468   6.74983 -1.32822 -0.71044 -1.24655 -0.69208] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [240]:
predict('망했네 학습;;;')

>> 입력하신 내용에서 슬픔이 느껴집니다. 

 [-0.67362 -1.58727 -0.67065 -1.5059  -1.73673  7.1885  -1.34239] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [241]:
predict('날씨가 너무 좋아')

>> 입력하신 내용에서 중립이 느껴집니다. 

 [-0.60715 -0.95209 -1.07924 -1.01745  6.64206 -1.4406  -1.1621 ] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [242]:
predict('마스크 너무 불편하다')

>> 입력하신 내용에서 두려움이 느껴집니다. 

 [-1.22248 -1.72993 -1.81096  6.04014 -1.72813 -0.21684 -1.21156] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [243]:
predict('집에 가고 싶다')

>> 입력하신 내용에서 슬픔이 느껴집니다. 

 [-0.28131 -1.30538 -1.31719 -1.20733 -2.05308  7.14587 -1.63598] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [244]:
predict('오늘 날씨가 너무 맑다')

>> 입력하신 내용에서 중립이 느껴집니다. 

 [-0.78147 -1.22275 -1.12496 -0.97342  6.70194 -1.25184 -0.85256] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [245]:
predict('친구들이 바빠서 얼굴보기 힘들어...')

>> 입력하신 내용에서 분노가 느껴집니다. 

 [ 4.13042 -2.30742 -2.6751   2.92396 -1.47471 -0.61004 -1.7572 ] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [246]:
predict('친구들이 바빠서 얼굴보기 힘들어')

>> 입력하신 내용에서 분노가 느껴집니다. 

 [ 4.25893 -2.58701 -2.69004  2.79063 -1.31538 -0.53067 -1.70671] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |


In [247]:
predict('너무 기쁘다')

>> 입력하신 내용에서 행복이 느껴집니다. 

 [-0.37456  6.73634 -0.68043 -1.51357 -0.96388  0.09943 -1.33746] 
 |   분노  |   행복  |   혐오  |   두려움  |   중립  |   슬픔  |  놀람   |
